In [1]:
import tensorflow.compat.v1 as tf
import warnings

tf.disable_v2_behavior()
tf.set_random_seed(777)  # for reproducibility

warnings.filterwarnings('ignore')

print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
2.4.0


In [2]:
# breast-cancer-wisconsin-data : https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

#define filename queue
filename_queue = tf.train.string_input_producer(['data-exercise.csv'],shuffle=False,name='filename_queue')

# define reader
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = []
for i in range(9):
    record_defaults.append([0.])

xy = tf.decode_csv(value, record_defaults=record_defaults)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.


In [3]:
# collect batches of csv
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], [xy[-1]]], batch_size=10)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


In [4]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [5]:
# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

# automatically differentiate the cost function while getting the minimum cost
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)

In [6]:
# Accuracy computation
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [7]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [8]:
# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [9]:
 for step in range(10001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, _ = sess.run([cost, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 2000 == 0:
        print(step, cost_val)

        #Accuracy report
        h, c, a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: x_batch, Y: y_batch})
        print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)

0 0.29224738

Hypothesis:  [[0.2584871 ]
 [0.23836318]
 [0.24619922]
 [0.26769924]
 [0.24774647]
 [0.25374082]
 [0.24306804]
 [0.2518505 ]
 [0.25321805]
 [0.26775503]] 
Correct (Y):  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] 
Accuracy:  1.0
2000 0.7103228

Hypothesis:  [[0.6039278 ]
 [0.6168612 ]
 [0.6057091 ]
 [0.61930615]
 [0.61881   ]
 [0.6216917 ]
 [0.61110085]
 [0.61680907]
 [0.6235681 ]
 [0.6136892 ]] 
Correct (Y):  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  0.5
4000 0.59168595

Hypothesis:  [[0.6305999 ]
 [0.62824893]
 [0.5878193 ]
 [0.6412373 ]
 [0.64560187]
 [0.6468078 ]
 [0.6307311 ]
 [0.5988451 ]
 [0.64136815]
 [0.64190376]] 
Correct (Y):  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  0.7
6000 0.90016377

Hypothesis:  [[0.6157086 ]
 [0.56659096]
 [0.5455355 ]
 [0.5147437 ]
 [0.5782331 ]
 [0.5830693 ]
 [0.6428324 ]
 [0.6074272 ]
 [0.63742197]
 [0.6100963 ]] 
Correct (Y):  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]